## Convolutional Networks

卷积神经网络(CNN) 是一种用来处理含有网格状拓扑结构(topology)的数据的神经网络. 

> 例如时间序列(time series)数据, 可以看做是固定时间间隔上的 1-D 网格; 而图像数据是像素的 2-D 网格.

**卷积(Convolutional)** 是一种特殊的线性(数学)操作.

> Def.
>
> 卷积网络是至少有一层被卷积来取代一般矩阵乘法的简单神经网络.

因为 Fully Connected MLP 会有大量的 parameters, 这很容易造成过拟合.

## convolution operation

$$s(t) = \int x(a) w(t - a) \mathsf{d}a$$

离散情况

$$s(t) = \sum_{a = - \infty}^\infty x(a) w(t-a)$$

denoted as $s(t) = (x * w)(t)$

> 实践中一般都是有限集, 并且 $x$ 一般是多维数组的数据(e.g., 三维张量), $w$ 一般是参数的多维数组, 所在在没有数据的地方都定义函数值为 $0$, 这样无限空间上的积分一般是有限集上的积分, $t$ 也一般是多维的.

$s(t)$ 是对 $x(t)$ 在加上概率分布 $w(x)$ 作为权重函数的平滑版的近似. 其中 $x$ 叫做 **输入 input**, 而 $w$ 叫做 **卷积核 convolution kernel 或者 filter  过滤器**, 而输出也叫做 **feature map**.

e.g.

对于二维图像 $I$ 作为 input, 以及二维 kernel $K$:

$$S(i, j) = (I * K) (i,j) = \sum_m \sum_n I(m,n)K(i - m, j -n )$$

Convolution 是 **可交换 commutative** 的, 所以我们还可以写作

$$S(i, j) = (K*I) (i,j) = \sum_m \sum_n K(m,n)I(i - m, j -n )$$

> [commutative Pf.](http://www.songho.ca/dsp/convolution/convolution_commutative.html) 主要是应用了 kernel 中减号的翻转(flip)

一般 library 中实现的是 **cross-correlation** (没有前面的那种 flipped 了)

$$S(i,j) = (K*I)(i,j) = \sum_m \sum_n I(i+m, j+n) K(m,n)$$

> 一般 convolution 都是跟其他函数一起用的, 所以是否 flip 一般不影响结果

kernel matrix 一般会限制某些元素等于其他元素, e.g., **Toeplitz matrix** 限制每一行是上一行移动一个元素之后的结果, 二维情况下, doubly circuland matrix 就是一种两重循环的 tensor. [Ref](https://dsp.stackexchange.com/questions/35373/convolution-as-a-doubly-block-circulant-matrix-operating-on-a-vector)

> 像前面 kernel 覆盖之外的区域可以视作权重是 $0$ 以及 kernel 限制为 Toeplitz matrix 这些都是 **无限强的先验信念 infinitely strong prior (超级低的熵)**

Toeplitz matrix

![](./images/cir_matrix.png)

下图描述了一个 2-D convolution without kernel flipping

![](./images/conv.png)

其实 CNN 就是一个 local conv 而不是 fully connected( **简写为 FC层** ) 的 feedforward net.

![](./images/local_conv.png)

> kernel 一般是比 input 小很多的 **稀疏矩阵 sparse matrix**.


## Conv.(convolution 的缩写) 的三个重要思想

* 稀疏连接 sparse interactions/sparse connecteivity/sparse weights (反正就是相对于 fully connected)

这样可以接受变长输入 inputs of variable size, 并且能够极大地减少空间和时间的消耗.

这样做在于, 可能输入图像(e.g.)有百万个像素, 而其中的一条边可能只是几百个像素, 这个可以称为图像的 **局部性原理(Locality)**.

某一 layer 中与下一 layer 某 unit 连接的 units 称为这个 unit 的 **接受域 receptive field**, 越深层次的 unit 的 (indirect)接受域就越大, 可以解释为越深层次的抽象能力越高.

* 参数共享 parameter sharing/tied weights

传统 NN 不会共享 parameters, 在 CNN 中, 因为 kernel matrix 比较小, 所以这一层的所有像素都用着一个 kernel matrix 来处理(跨越多大步长(**stride** )来处理整个图像是一个超参数(就有点像盖章盖满一张白纸, 如何选择每一个章之间的距离就是这个步长, 并且在边界情况下可能盖不全, 这时候可以盖部分或者用 **zero-padding** 用 $0$ 填充), 能够减少空间消耗.

如果没有 zero-padding 并且使用 **valid convolution**(matlab 术语(terminology), 下同), 会出现输出 **shrinkage 缩小** 的结果, 这样会限制 conv layer 层数进而限制模型表达

> 添加 zero-padding 的地方往往需要更加大一点的 biases

![](./images/shrinkage.png)

如果加上 zero-padding 就不会出现这种情况, 这种情况叫做 **same convolution**

![](./images/zero_padding.png)

还有一种 zero-padding 多加的情况, 会导致输出比输入大, 这种情况叫做 **full convolution**, 不过这种会导致 kernel 的优化很难, 不怎么用.


* 等变表示 equivariant representation

因为参数共享, 所以有等变表示. 这点很有用处, 因为图像中的物体往往可以移动其位置, 这样就算物体移动了位置, 因为参数共享, 我们依然能够得到相同的特征. 不过这种等变表示只适用于移动变换, 如果是缩放或者变形或者旋转变换都没办法处理. 而且这种优势在某些应用上可能会变成劣势, 比如人脸识别中有物体之间的相对位置要求, 比如(一张正放的人物照片), 左右两只眼睛是不一样的.

## Pooling 池化

一层典型的 CNN 层一般由三种 **阶段 stage**(也可以理解为 layer) 组成

* Conv. stage 执行 affine transformation
* Detector stage 检测阶段, 执行 nonlinearity (e.g. ReLU)
* Pooling stage, 将输出元素用它周围的一片局域的 **统计结果 summary statistic** 来替代, e.g. **max polling** 就是以它为中心的一个矩形区域的最大值来代替它, 还有其他诸如求矩形区域的 $L^2$ 范数. 这样的用处: 确保输入的少量变换的结果 **invariant 不变性**, 如果我们需要确保特征并不一定要准确的出现在一个确定的位置的时候, 总的来说就是提高了泛化能力(也这算是一个先验 piror). 跟前面参数共享中说的那个步长 stride 一样, pooling 也可以通过增加步长来 **downsample 减少采样**, 这样可以减少输出量进而减少计算量, 

stride 为 2 的 pooling

![](./images/downsample.png)

pooling 可用于接受变长输入, 可以在 pooling 层动态的改变 pooling 大小或者步长大小对不同大小输入产生相同输出就好.

pooling 也可以视作是 infinitely strong prior, 这些 convolution 和 pooling 的 prior 如果设置不正确(比如那些先验假设不适用于当前训练数据集), 就很有可能产生 **欠拟合(underfitting)**


## Variants of convolution function

实际中, 因为一种 convolution function 只能提取一种特征, 如果我们需要在一个 layer 中提取多种特征, 我们就需要多种 conv. operation, 每种负责一部分输入和输出.  而且一般的输入都不是矩阵而是一个有深度 depth (e.g., 图片一般有 RGB 三层信息组成, 所以一般是 width * height * depth 的 tensor) 的张量. 就像下图

![](./images/local_conv.png)

> tensor 的乘法往往是 reshape 成 matrix 在进行乘法

而且一般的软件实现是 **batch 批处理** 而不是单个样本, 所以一般是 4-D tensor(最后一维就是样本的 index)

所以 conv 层一般是 multichannel conv., conv. operation 就是对 multichannel 进行处理的而不是原来的一个 channel, 所以 conv. operation 的可交换性就失效了, **除非这个 conv. operation 的输入和输出的 channels 数是一样的** (一般的 ConvNet 也是这样要求的)

前面提到的 stride 步长的不同也是一种变种.

还有一种情况下我们只想从特定局部位置去提取特征, e.g., 我们要想保证不同区域的眼睛是不一样的(左眼和右眼), 这样可以像 conv 一样使用一个比较小的 kernel matrix, 但是每一局部的 kernel matrix 都不一样(i.e., 不共享), 不会像 convolution 一样 share parameters(kernel matrix), 这种称为 **locally connected layer** 或 **unshare convolution**.

![](./images/unshare_conv.png)

> 上面的是 unshare conv, 下面的是 conv.

还有一种介于 unshare conv. 和 conv. 之间的, 叫做 **Tiled conv.**

![](./images/tiled_conv.png)

> 上面是 unshare conv., 下面是 tiled conv., tiled conv. 相当于多种 kernel matrix 成块的 conv., 这样相对于 ushare conv. 可以减少空间占用.

**Dilated convolutions 膨胀卷积** 进行卷积的时候会有一个间隔的选取元素组成矩阵再乘以卷积核, 

e.g., in one dimension a filter w of size 3 would compute over input x the following: w\[0\]\*x\[0\] + w\[1\]\*x\[1\] + w\[2\]\*x\[2\]. 

This is dilation of 0. For dilation 1 the filter would instead compute w\[0\]\*x\[0\] + w\[1\]\*x\[2\] + w\[2\]\*x\[4\]

## Structured Outputs

